In [1]:
import os
import glob

from bs4 import BeautifulSoup
from openpyxl import load_workbook
import pandas as pd
import requests
import wget
import urllib.request

%matplotlib inline
import seaborn as sns; sns.set()
import matplotlib as mpl
import matplotlib.pyplot as plt
#Enlargen
sns.set_context('poster')

plt.style.use('fivethirtyeight')

In [2]:
r = requests.get('https://www.colorado.gov/pacific/revenue/marijuana-tax-data-archive')
soup = BeautifulSoup(r.text, "lxml")

anchor_tags = soup.findAll('div','fieldset-wrapper')[1].findAll('a')

links = [tag['href'] for tag in anchor_tags]

In [3]:
def download_if_needed(link):
    """
    Download pdf file at link if not already downloaded, returning the filename
    """
    filename = os.path.basename(urllib.request.unquote(link))
    if not os.path.exists('excels/{}'.format(filename)):
        #wget needs percent encoding to go away
        wget.download(urllib.request.unquote(link), out='excels/')
    return filename

In [4]:
def get_year_month(filename):
    date_code = os.path.basename(filename)[:4]
    date_code = '20{year}-{month}'.format(year=date_code[2:4],month=date_code[0:2])
    return date_code

In [5]:
def create_df_from_file(file):
    wb = load_workbook(file)
    ws = wb.get_sheet_by_name(wb.get_sheet_names()[0])
    
    df = pd.DataFrame()

    row = 6
    while row < 100:

        county = ws['A{}'.format(row)].value

        if 'Totals' in county:
            break
        if 'Remainder of' in county:
            county = 'Other Counties'
        if county.lower() == 'route':
            county = 'Routt'
        df.loc[county,'Medical Marijuana'] = ws['B{}'.format(row)].value
        row +=1
    
    row = 6
    while row < 100:
        county = ws['D{}'.format(row)].value

        if 'Totals' in county:
            break
        if 'Remainder of' in county:
            county = 'Other Counties'
        if county.lower() == 'route':
            print('Success')
            county = 'Routt'
        df.loc[county,'Retail Sales Tax'] = ws['E{}'.format(row)].value
        df.loc[county,'Additional Retail Sales Tax'] = ws['F{}'.format(row)].value

        row +=1
        
    df.index.name = 'County'
    
    df['Month'] = get_year_month(file.name)
    df['Month'] = pd.DatetimeIndex(df['Month'])
    
    return df

In [6]:
dfs = []

for link in links:
    filename = download_if_needed(link)
    
    with open('excels/{}'.format(filename),'rb') as file:
        df = create_df_from_file(file)
        dfs.append(df)
    
df = pd.concat(dfs)
df.fillna(0, inplace=True)

//anaconda/lib/python3.5/site-packages/openpyxl/workbook/names/named_range.py:124: UserWarning: Discarded range with reserved name
  warnings.warn("Discarded range with reserved name")


Success


In [7]:
df.head()

,Medical Marijuana,Retail Sales Tax,Additional Retail Sales Tax,Month
County,,,,
Adams,8304,13092,29053,2015-01-01
Arapahoe,25572,53858,135531,2015-01-01
Boulder,64260,109224,323530,2015-01-01
Clear Creek,2117,12583,35220,2015-01-01
Denver,363061,423513,1241631,2015-01-01


In [8]:
df.pivot(columns='Month',values='Medical Marijuana').fillna(0).to_csv('Medical_time_series.csv')
df.pivot(columns='Month',values='Retail Sales Tax').fillna(0).to_csv('Retail_time_series.csv')
df.pivot(columns='Month',values='Additional Retail Sales Tax').fillna(0).to_csv('Additional_retail_time_series.csv')


In [9]:
df = df.reset_index().set_index(['County','Month'])

In [10]:
total_df = pd.DataFrame(df.sum(axis=1))

In [11]:
total_df.reset_index(inplace=True)

In [12]:
total_df.columns

Index(['County', 'Month', 0], dtype='object')

In [13]:
total_df.pivot(index='County',columns='Month',values=0).fillna(0).to_csv('Total_time_series.csv')